In [5]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [6]:
def train_test(X, p=0.8):
    np.random.shuffle(X)
    cant = np.ceil(len(X) * p)
    return X[:int(cant)], X[int(cant):]

In [7]:
data = pd.read_csv('data.csv')
train, test = train_test(data.to_numpy())
X_train, y_train = train[:, :-1], train[:, -1]
X_test, y_test = test[:, :-1], test[:, -1]

print(len(data), len(train), len(test))

500 400 100


In [ ]:
# k-fold cross-validation
from sklearn.model_selection import cross_val_score, cross_validate
k = 5
scores = cross_validate(DecisionTreeClassifier(max_depth=3), X_train, y_train, cv=k, return_train_score=True)
print(scores)

# cross val with AUCROC
from sklearn.metrics import make_scorer, roc_auc_score
aucroc_scorer = make_scorer(roc_auc_score)
aucroc_scores = cross_validate(DecisionTreeClassifier(max_depth=3), X_train, y_train, cv=k, scoring=aucroc_scorer, return_train_score=True)

# cross val with AUPRC
from sklearn.metrics import average_precision_score
auprc_scorer = make_scorer(average_precision_score)
auprc_scores = cross_validate(DecisionTreeClassifier(max_depth=3), X_train, y_train, cv=k, scoring=auprc_scorer, return_train_score=True)

print(f"Cross-validation scores (AUCROC): {aucroc_scores}")
print(f"Cross-validation scores (AUPRC): {auprc_scores}")
print(f"Cross-validation scores: {scores}")
# means
print(f"Cross-validation mean score: {scores.mean()}")
print(f"Cross-validation mean AUCROC score: {aucroc_scores.mean()}")
print(f"Cross-validation mean AUPRC score: {auprc_scores.mean()}")

{'fit_time': array([0.03000093, 0.02900004, 0.0370028 , 0.02599859, 0.02599859]), 'score_time': array([0.00199652, 0.00099874, 0.0010004 , 0.00100017, 0.0010004 ]), 'test_score': array([0.675 , 0.7   , 0.7   , 0.5   , 0.6375])}
Cross-validation scores (AUCROC): {'fit_time': array([0.02499819, 0.02604032, 0.02699828, 0.02599835, 0.02401662]), 'score_time': array([0.00200081, 0.00196242, 0.00203109, 0.00200129, 0.00199986]), 'test_score': array([0.61090909, 0.62909091, 0.57636364, 0.43803419, 0.5519943 ])}
Cross-validation scores (AUPRC): {'fit_time': array([0.02699971, 0.02696705, 0.0290029 , 0.02899551, 0.02599907]), 'score_time': array([0.00100088, 0.00100493, 0.0040009 , 0.00100088, 0.00100064]), 'test_score': array([0.38543478, 0.40547619, 0.365     , 0.30738866, 0.35455466])}
Cross-validation scores: {'fit_time': array([0.03000093, 0.02900004, 0.0370028 , 0.02599859, 0.02599859]), 'score_time': array([0.00199652, 0.00099874, 0.0010004 , 0.00100017, 0.0010004 ]), 'test_score': array

AttributeError: 'dict' object has no attribute 'mean'

In [14]:
criterias = ["gini", "entropy"]
alturas = [3, 5, None]

for criteria in criterias:
    for h in alturas:
        res = cross_validate(DecisionTreeClassifier(max_depth=h, criterion=criteria), X_train, y_train, cv=k, return_train_score=True)
        print(criteria, h)
        print(f'Train score: {res["train_score"].mean()}, Val score {res["test_score"].mean()}')


gini 3
Train score: 0.8193750000000002, Val score 0.6449999999999999
gini 5
Train score: 0.9275, Val score 0.6525000000000001
gini None
Train score: 1.0, Val score 0.6075
entropy 3
Train score: 0.7918749999999999, Val score 0.64
entropy 5
Train score: 0.890625, Val score 0.655
entropy None
Train score: 1.0, Val score 0.6199999999999999


Al dejar altura infinita, overfittea (un poco). No hay casi diferencia entre Gini y entropia


(320, 200) (320,)
(320, 200) (320,)
(320, 200) (320,)
(320, 200) (320,)
(320, 200) (320,)
Cross-validation scores (custom): [[0.66454545 0.54761905]
 [0.79415954 0.5       ]
 [0.72213895 0.49820789]
 [0.81317702 0.52909091]
 [0.79041488 0.61942175]]
Cross-validation mean score (custom): [0.75688717 0.53886792]
